In [1]:
import numpy as np
import time


index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score
import numpy as np
from scipy.sparse import vstack

def stratified_random_sample(X, y, fraction=None, count=None, random_state=42):
    assert (fraction is not None) ^ (count is not None), "Specify either fraction or count"

    rng = np.random.RandomState(seed=random_state)
    X_out, y_out = [], []

    for label in np.unique(y):
        X_c = X[y == label]
        y_c = y[y == label]

        n_total = len(X_c)
        n_sample = int(n_total * fraction) if fraction is not None else min(count, n_total)
        if n_sample == 0:
            n_sample = 1

        idx = rng.choice(n_total, size=n_sample, replace=False)

        X_out.append(X_c[idx])
        y_out.append(y_c[idx])

    return np.concatenate(X_out, axis=0), np.concatenate(y_out, axis=0)


def classwise_density_weighted_sampling_by_fraction(X_pool, y_pool, model, fraction=0.2, random_state=42):
    X_selected = []
    y_selected = []
    selected_indices = []

    classes = np.unique(y_pool)
    probs = model.predict_proba(X_pool)
    rng = np.random.RandomState(seed=random_state)

    # Initialize NearestNeighbors for density calculation
    nn = NearestNeighbors(n_neighbors=5)
    nn.fit(X_pool)

    for c in classes:
        idx_c = np.where(y_pool == c)[0]
        X_c = X_pool[idx_c]
        probs_c = probs[idx_c]

        n_c = len(idx_c)
        n_select = max(1, int(n_c * fraction))

        if n_c == 0:
            continue

        # Calculate uncertainty: 1 - max(probability)
        uncertainties = 1 - np.max(probs_c, axis=1)  # shape: (n_samples_in_class,)

        # Calculate density: inverse of sum of distances to k-nearest neighbors
        distances = nn.kneighbors(X_c, n_neighbors=5, return_distance=True)[0]
        density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)

        # Combine uncertainty and density
        scores = uncertainties * density

        # Select top-k samples with highest scores
        sorted_idx = np.argsort(scores)[-n_select:]
        top_k_idx = idx_c[sorted_idx]

        selected_indices.extend(top_k_idx)
        X_selected.append(X_pool[top_k_idx])
        y_selected.append(y_pool[top_k_idx])

    selected_indices = np.array(selected_indices)
    X_selected = vstack(X_selected).toarray() if len(X_selected) > 0 else np.array([])
    y_selected = np.concatenate(y_selected) if len(y_selected) > 0 else np.array([])

    # Remaining data
    remaining_idx = np.setdiff1d(np.arange(len(X_pool)), selected_indices)
    X_rest = X_pool[remaining_idx]
    y_rest = y_pool[remaining_idx]

    # Print selected samples' label distribution
    print(f"Selected samples (fraction={fraction}): {len(y_selected)}")
    print(f"Label distribution: {np.bincount(y_selected, minlength=11)}")

    return X_rest, X_selected, y_rest, y_selected


# Example usage
X_pool = x_test_adv_0_1  # Your data
y_pool = y_label_0_1     # Your labels (0 to 20)

# Initial stratified sampling
X_train_init, y_train_init = stratified_random_sample(X_pool, y_pool, fraction=0.1, random_state=42)
print(f"Initial labeled set size: {len(y_train_init)}")
print(f"Initial label distribution: {np.bincount(y_train_init, minlength=11)}")

# Train initial model
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_init, y_train_init)

_, X_20, _, y_20 = classwise_density_weighted_sampling_by_fraction(
    x_test_adv_0_1, y_label_0_1, clf, fraction=0.2, random_state=42
)

Initial labeled set size: 102570
Initial label distribution: [75542   161   700  1743   377 10542    74   736 12435   172    88]


/tmp/ipykernel_1035/3449763556.py:59: RuntimeWarning: divide by zero encountered in divide
  density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)
/tmp/ipykernel_1035/3449763556.py:62: RuntimeWarning: invalid value encountered in multiply
  scores = uncertainties * density
/tmp/ipykernel_1035/3449763556.py:59: RuntimeWarning: divide by zero encountered in divide
  density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)
/tmp/ipykernel_1035/3449763556.py:59: RuntimeWarning: divide by zero encountered in divide
  density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)
/tmp/ipykernel_1035/3449763556.py:62: RuntimeWarning: invalid value encountered in multiply
  scores = uncertainties * density
/tmp/ipykernel_1035/3449763556.py:59: RuntimeWarning: divide by zero encountered in divide
  density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)
/tmp/ipykernel_1035/3449763556.py:59: RuntimeWarning: divide by zero encountered in di

Selected samples (fraction=0.1): 102570
Label distribution: [75542   161   700  1743   377 10542    74   736 12435   172    88]


In [6]:
X_20.shape

(102570, 56)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb20 = XGBClassifier()
xgb20.fit(X_20, y_20)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf20 = RandomForestClassifier()
rf20.fit(X_20, y_20)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt20 = DecisionTreeClassifier()
dt20.fit(X_20, y_20)

RF


RandomForestClassifier()

In [10]:
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [8]:
import numpy as np



base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb20.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ActiveLearning_DensityWeighted/UNSW_Input20/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ActiveLearning_DensityWeighted/UNSW_Input20/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

In [12]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf20.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning_DensityWeighted/UNSW_Input20/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning_DensityWeighted/UNSW_Input20/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  8  9 10] [   49  1472   918  9430   484  8543 60858 29083  3134]
(49, 56) (49,)
Save 0 to baseline_Def1.npy
(1472, 56) (1472,)
Save 1 to baseline_Def2.npy
(918, 56) (918,)
Save 2 to baseline_Def3.npy
(9430, 56) (9430,)
Save 3 to baseline_Def4.npy
(484, 56) (484,)
Save 4 to baseline_Def5.npy
(8543, 56) (8543,)
Save 5 to baseline_Def6.npy
(60858, 56) (60858,)
Save 8 to baseline_Def9.npy
(29083, 56) (29083,)
Save 9 to baseline_Def10.npy
(3134, 56) (3134,)
Save 10 to baseline_Def11.npy
Execution Time: 1.005217 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [ 56568    752   2622  37767   5588 204679      1   8662  22627   2647]
(56568, 56) (56568,)
Save 0 to BIM_Def1.npy
(752, 56) (752,)
Save 1 to BIM_Def2.npy
(2622, 56) (2622,)
Save 2 to BIM_Def3.npy
(37767, 56) (37767,)
Save 3 to BIM_Def4.npy
(5588, 56) (5588,)
Save 4 to BIM_Def5.npy
(204679, 56) (204679,)
Save 5 to BIM_Def6.npy
(1, 56) (1,)
Save 7 to BIM_Def8.npy
(8662, 56) (8662,)
Save 8 to BIM_Def9.npy


In [ ]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt20.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ActiveLearning_DensityWeighted/UNSW_Input20/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ActiveLearning_DensityWeighted/UNSW_Input20/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

In [13]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}20")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_ActiveLearning_DensityWeighted/UNSW_Input20/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [14]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input20_ActiveLearning_DensityWeighted.csv")

In [15]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
1,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
2,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
3,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
4,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
5,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
6,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
7,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
8,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
9,XGB10,0,0,0,0,0,0,0,0,0,0,0,0
